# Домашняя работа 1

### Задание 1. Создание структуры базы данных

In [87]:
# импорт библиотек
import pandas as pd
import numpy as np

![image_1](img/img_1.png)

### Задание 2. Исследование и нормализация данных

In [58]:
# чтение данных по транзакциям

transactions = pd.read_csv('customer_and_transaction/transaction-Tаблица 1.csv', sep=';')
transactions.head(2)

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost
0,1,2,2950,2/25/17,False,Approved,Solex,Standard,medium,medium,"71,49","53,62"
1,2,3,3120,5/21/17,True,Approved,Trek Bicycles,Standard,medium,large,"2091,47","388,92"


In [59]:
# чтение данных по пользователям

customers = pd.read_csv('customer_and_transaction/customer-Tаблица 1.csv', sep=';')
customers.head(2)

,customer_id,first_name,last_name,gender,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,address,postcode,state,country,property_valuation
0,1,Laraine,Medendorp,F,1953-10-12,Executive Secretary,Health,Mass Customer,N,Yes,060 Morning Avenue,2016,New South Wales,Australia,10
1,2,Eli,Bockman,Male,1980-12-16,Administrative Officer,Financial Services,Mass Customer,N,Yes,6 Meadow Vale Court,2153,New South Wales,Australia,10


In [60]:
# создание таблицы по товарам

goods = transactions[[
    'product_id',
    'brand',
    'product_line',
    'product_class',
    'product_size',
    'list_price',
    'standard_cost'
]]

Исследуем данные в таблице **goods**. Данные ненормализованы.

In [61]:
# поиск дубликатов строк

n_rows = goods.shape[0]
n_rows_wo_duplicates = goods.drop_duplicates().shape[0]

print(f'Количество строк в исходной таблице - {n_rows}')
print(f'Количество строк после удаления всех дубликатов - {n_rows_wo_duplicates}')


Количество строк в исходной таблице - 20000
Количество строк после удаления всех дубликатов - 397


Данные не находятся в 1НФ из-за наличия дубикатов

In [62]:
# удаление дубликатов

goods = goods.drop_duplicates()

Для сведения данных к 2НФ найдем ключевые поля. Для этого найдем уникальные элементы, определенные единственным образом, которые можем использовать в качестве первичного ключа.

In [63]:
n_rows = goods.shape[0]
n_products = goods.product_id.nunique()
n_brands = goods.brand.nunique()
n_product_line = goods.product_line.nunique()
n_product_class = goods.product_class.nunique()
n_product_size = goods.product_size.nunique()
n_list_price = goods.list_price.nunique()
n_standard_cost = goods.standard_cost.nunique()

print(f'Количество уникальных строк - {n_rows}')
print(f'Количество уникальных продуктов - {n_products}')
print(f'Количество брендов - {n_brands}')
print(f'Количество продуктовых линеек - {n_product_line}')
print(f'Количество классов продуктов - {n_product_class}')
print(f'Количество размеров продуктов - {n_product_size}')
print(f'Количество уникальных значений list price - {n_list_price}')
print(f'Количество уникальных значений standard cost - {n_standard_cost}')

Количество уникальных строк - 397
Количество уникальных продуктов - 101
Количество брендов - 6
Количество продуктовых линеек - 4
Количество классов продуктов - 3
Количество размеров продуктов - 3
Количество уникальных значений list price - 296
Количество уникальных значений standard cost - 100


In [64]:
# проверка на наличие пустых строк

goods.isna().sum()

product_id         0
brand            196
product_line     196
product_class    196
product_size     196
list_price         0
standard_cost    196
dtype: int64

Ни одно из указанных полей не может быть первичным ключом из-за несоответствия количества строк количеству уникальных элементов полей, а также из-за наличия пропусков в данных. Для приведения данных к 2НФ создадим поле **goods_id** целого типа, которое будет уникальным первичным ключом.

In [65]:
goods['goods_id'] = np.arange(1, len(goods)+1)
goods.head()

,product_id,brand,product_line,product_class,product_size,list_price,standard_cost,goods_id
0,2,Solex,Standard,medium,medium,"71,49","53,62",1
1,3,Trek Bicycles,Standard,medium,large,"2091,47","388,92",2
2,37,OHM Cycles,Standard,low,medium,"1793,43","248,82",3
3,88,Norco Bicycles,Standard,medium,medium,"1198,46","381,10",4
4,78,Giant Bicycles,Standard,medium,large,"1765,3","709,48",5


Ввиду малочисленности уникальных данных в столбцах **brand**, **product_line**, **product_class**, **product_size** нецелосообразно приводить данные к 3НФ для удаления транзитивности признаков.

In [66]:
# сохранение данных в .csv

goods.to_csv('customer_and_transaction/goods.csv', index=False)

Исследуем таблицу **transctions** на нормализованность. 

In [67]:
# Проверим количество дубликатов

n_rows = transactions.shape[0]
n_rows_wo_duplicates = transactions.shape[0]

print(f'Количество строк в исходной таблице - {n_rows}')
print(f'Количество строк после удаления всех дубликатов - {n_rows_wo_duplicates}')

Количество строк в исходной таблице - 20000
Количество строк после удаления всех дубликатов - 20000


Данные в 1НФ. Проверим соответствие данных 2НФ

In [68]:
# Проверка transaction_id на уникальность

print(f'Количество уникальных строк - {n_rows}')
print(f'Количество уникальных элементов transaction_id - {transactions.transaction_id.nunique()}')

Количество уникальных строк - 20000
Количество уникальных элементов transaction_id - 20000


Поле transaction_id является уникальным, поэтому он может быть первичным ключом. Данные в 2 НФ. Для приведения данных в 3НФ необходимо создать поле goods_id, внешний ключ к таблице **goods**, так **product_id** не соблюдает требованиям предъявляемым к ключам

In [69]:
# Объединение таблиц transaction, goods

transactions_merged = transactions.merge(
    goods,
    left_on=[
        'product_id',
        'brand',
        'product_line',
        'product_class',
        'product_size',
        'list_price',
        'standard_cost'
    ], right_on=[
        'product_id',
        'brand',
        'product_line',
        'product_class',
        'product_size',
        'list_price',
        'standard_cost'
    ]
)

transactions_merged.head()

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,goods_id
0,1,2,2950,2/25/17,False,Approved,Solex,Standard,medium,medium,"71,49","53,62",1
1,2,3,3120,5/21/17,True,Approved,Trek Bicycles,Standard,medium,large,"2091,47","388,92",2
2,3,37,402,10/16/17,False,Approved,OHM Cycles,Standard,low,medium,"1793,43","248,82",3
3,4,88,3135,8/31/17,False,Approved,Norco Bicycles,Standard,medium,medium,"1198,46","381,10",4
4,5,78,787,10/1/17,True,Approved,Giant Bicycles,Standard,medium,large,"1765,3","709,48",5


In [70]:
# Проверка количества внешних ключей - 397
transactions_merged.goods_id.nunique() == 397

True

In [71]:
# Проверка количества строк - 20000
transactions_merged.shape[0] == 20000

True

In [72]:
# Проверка консистентности данных

cond_1 = transactions_merged[transactions_merged.goods_id == 100][
        ['product_id',	'brand', 'product_line', 'product_class', 'product_size', 'list_price',	'standard_cost']].reset_index(drop=True)
cond_2 = goods[goods.goods_id == 100][
        ['product_id',	'brand', 'product_line', 'product_class', 'product_size', 'list_price',	'standard_cost']].reset_index(drop=True)
cond_1.loc[0]

product_id                   67
brand            Norco Bicycles
product_line               Road
product_class            medium
product_size             medium
list_price               544,05
standard_cost            376,84
Name: 0, dtype: object

In [73]:
cond_2

,product_id,brand,product_line,product_class,product_size,list_price,standard_cost
0,67,Norco Bicycles,Road,medium,medium,"544,05","376,84"


In [74]:
# удаление столбцов

transactions_merged.drop(
    columns=['product_id', 'brand', 'product_line', 'product_class', 'product_size', 'list_price',	'standard_cost'],
    inplace=True
)
transactions_merged.head()

,transaction_id,customer_id,transaction_date,online_order,order_status,goods_id
0,1,2950,2/25/17,False,Approved,1
1,2,3120,5/21/17,True,Approved,2
2,3,402,10/16/17,False,Approved,3
3,4,3135,8/31/17,False,Approved,4
4,5,787,10/1/17,True,Approved,5


In [75]:
# сохранение данных в transaction.csv

transactions_merged.to_csv('customer_and_transaction/transactions.csv', index=False)

Проверим на нормальность таблицу **customers**.

In [76]:
# Поиск дубликатов
n_rows = customers.shape[0]
n_rows_wo_duplicates = customers.drop_duplicates().shape[0]


print(f'Количество строк в исходной таблице - {n_rows}')
print(f'Количество строк после удаления всех дубликатов - {n_rows_wo_duplicates}')

Количество строк в исходной таблице - 4000
Количество строк после удаления всех дубликатов - 4000


In [77]:
# Разобьем поле address на 2 поля поле address

customers['street'] = customers.address.apply(lambda x: ' '.join(x.split()[1::]))
customers['building'] = customers.address.apply(lambda x: int(x.split()[0]))

customers.drop(columns=['address'], inplace=True)

customers.head()

,customer_id,first_name,last_name,gender,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,postcode,state,country,property_valuation,street,building
0,1,Laraine,Medendorp,F,1953-10-12,Executive Secretary,Health,Mass Customer,N,Yes,2016,New South Wales,Australia,10,Morning Avenue,60
1,2,Eli,Bockman,Male,1980-12-16,Administrative Officer,Financial Services,Mass Customer,N,Yes,2153,New South Wales,Australia,10,Meadow Vale Court,6
2,3,Arlin,Dearle,Male,1954-01-20,Recruiting Manager,Property,Mass Customer,N,Yes,4211,QLD,Australia,9,Holy Cross Court,0
3,4,Talbot,NaN,Male,1961-10-03,NaN,IT,Mass Customer,N,No,2448,New South Wales,Australia,4,Del Mar Point,17979
4,5,Sheila-kathryn,Calton,Female,1977-05-13,Senior Editor,NaN,Affluent Customer,N,Yes,3216,VIC,Australia,9,Oakridge Court,9


Данные приведены в 1НФ

In [78]:
# Проверка customer_id на уникальность

print(f'Количество уникальных строк - {n_rows}')
print(f'Количество уникальных элементов customer_id - {customers.customer_id.nunique()}')

Количество уникальных строк - 4000
Количество уникальных элементов customer_id - 4000


Данные в 2НФ. Поле **customer_id** содержит только уникальные значения и является первичным ключом. Приведем данные в 3НФ. Поля **country**, **state**, **post_code**, **street**, **building** являются транзитивными. Их можно выделить отдельную таблицу **address**

In [79]:
# Создание таблицы address

address = customers[
    [
        'country',
        'state',
        'postcode',
        'street',
        'building'
    ]
]

address = address.drop_duplicates()
address.head()

,country,state,postcode,street,building
0,Australia,New South Wales,2016,Morning Avenue,60
1,Australia,New South Wales,2153,Meadow Vale Court,6
2,Australia,QLD,4211,Holy Cross Court,0
3,Australia,New South Wales,2448,Del Mar Point,17979
4,Australia,VIC,3216,Oakridge Court,9


Данные в 1НФ. Создадим поле **address_id** в качестве первичного ключа

In [80]:
# создание поля address_id

address['address_id'] = np.arange(1, len(address)+1)
address.head()

,country,state,postcode,street,building,address_id
0,Australia,New South Wales,2016,Morning Avenue,60,1
1,Australia,New South Wales,2153,Meadow Vale Court,6,2
2,Australia,QLD,4211,Holy Cross Court,0,3
3,Australia,New South Wales,2448,Del Mar Point,17979,4
4,Australia,VIC,3216,Oakridge Court,9,5


Данные в 2НФ. Нет транзитивных полей. Данные в 3НФ

In [81]:
# создание поля address_id в таблице customers и приведение ее к 3НФ

customers_merged = customers.merge(
    address,
    left_on=[
        'country',
        'state',
        'postcode',
        'street',
        'building'
    ], right_on=[
        'country',
        'state',
        'postcode',
        'street',
        'building'
    ]
)

customers_merged.head()

,customer_id,first_name,last_name,gender,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,postcode,state,country,property_valuation,street,building,address_id
0,1,Laraine,Medendorp,F,1953-10-12,Executive Secretary,Health,Mass Customer,N,Yes,2016,New South Wales,Australia,10,Morning Avenue,60,1
1,2,Eli,Bockman,Male,1980-12-16,Administrative Officer,Financial Services,Mass Customer,N,Yes,2153,New South Wales,Australia,10,Meadow Vale Court,6,2
2,3,Arlin,Dearle,Male,1954-01-20,Recruiting Manager,Property,Mass Customer,N,Yes,4211,QLD,Australia,9,Holy Cross Court,0,3
3,4,Talbot,NaN,Male,1961-10-03,NaN,IT,Mass Customer,N,No,2448,New South Wales,Australia,4,Del Mar Point,17979,4
4,5,Sheila-kathryn,Calton,Female,1977-05-13,Senior Editor,NaN,Affluent Customer,N,Yes,3216,VIC,Australia,9,Oakridge Court,9,5


In [82]:
# Проверка консистентности

customers_merged[customers_merged.address_id == 100][[
    'country',
    'state',
    'postcode',
    'street',
    'building'
]].reset_index(drop=True).loc[0]

country               Australia
state                       QLD
postcode                   4171
street      New Castle Crossing
building                  85688
Name: 0, dtype: object

In [83]:
address[address.address_id == 100]

,country,state,postcode,street,building,address_id
99,Australia,QLD,4171,New Castle Crossing,85688,100


In [84]:
customers_merged.shape[0] == n_rows

True

In [85]:
# удаление транзитивных полей
customers_merged.drop(
    columns=[
        'country',
        'state',
        'postcode',
        'street',
        'building'
    ], 
    inplace=True
)

customers_merged.head()

,customer_id,first_name,last_name,gender,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,property_valuation,address_id
0,1,Laraine,Medendorp,F,1953-10-12,Executive Secretary,Health,Mass Customer,N,Yes,10,1
1,2,Eli,Bockman,Male,1980-12-16,Administrative Officer,Financial Services,Mass Customer,N,Yes,10,2
2,3,Arlin,Dearle,Male,1954-01-20,Recruiting Manager,Property,Mass Customer,N,Yes,9,3
3,4,Talbot,NaN,Male,1961-10-03,NaN,IT,Mass Customer,N,No,4,4
4,5,Sheila-kathryn,Calton,Female,1977-05-13,Senior Editor,NaN,Affluent Customer,N,Yes,9,5


Для приведения к 3НФ необходимо убрать транзитивность в полях **job_title**, **job_industry_category**, но в данной работе это считается нецелесообразным из-за наличия пропусков в данных

In [86]:
# сохранение таблиц customers.csv и address.to_csv

customers_merged.to_csv('customer_and_transaction/customers.csv', index=False)
address.to_csv('customer_and_transaction/address.csv', index=False)

### Задание 3. Создание таблиц в DBeaver

```sql
drop table goods;
drop table customers cascade;
drop table customer_address cascade;
drop table transactions cascade;


create table goods (
	goods_id integer primary key
	,brand text
	,product_id integer
	,product_line varchar(20)
	,product_class varchar(20)
	,product_size varchar(20)
	,list_price float
	,standard_cost float
);

create table customers(
	customer_id integer primary key
	,first_name text
	,last_name text
	,gender varchar(20)
	,DOB date
	,job_title text
	,job_industry_category text
	,wealth_segment text
	,deceased_indicator varchar(20)
	,owns_car varchar(20)
	,property_valuation integer
	,address_id integer
	,constraint fk_customer_address
		foreign key (address_id)
			references customer_address(address_id)
);

create table customer_address (
	address_id integer primary key
	,postcode integer
	,country text
	,state text
	,street text
	,building integer
);

create table transactions (
	transaction_id integer primary key
	,goods_id integer
	,customer_id integer
	,transaction_date date
	,online_order boolean
	,order_status varchar(30)
	,constraint fk_goods
		foreign key (goods_id)
			references goods(goods_id)
	,constraint fk_customers
		foreign key (customer_id)
			references customers(customer_id)
); 


```

![image2](img/img_2.png)

### Задание 4. Загрузка данных в таблицы

![img_3](img/img_3.png)

**Таблица с адресами заказчиков**

![img_4](img/img_4.png)

**Таблица с данными заказчиков**

![img_5](img/img_5.png)

**Таблица с транзакциями**

![img_6](img/img_6.png)

**Таблица с товарами**